In [ ]:
def apply_box_deltas_graph(boxes, deltas):
    """
    Applies the given deltas to the given boxes.

    x,y,w,h : Bounding Box coordinates, width, and height

    Boxes:  is the (y1, x1, y2, x2) of the anchor boxes
    deltas: [dy, dx, log(dh), log(dw)] 
            is the predicted bounding box returned from the RPN layer (in test phase)
            These are considered the targets or ground truth 
    Refer to Bounding Box Regression - R-CNN paper 
    Regression targets are calculated as follows:   
      tx = (GTx - PRx)/PRw       ty = (Gy - Py)/Ph
      th = log(Gw/Pw)         tw = log(Gh/Ph)
    ---------------------------------------------------------------------------------            
    
    boxes:  [N, 4] where each row is [y1, x1, y2, x2]
    deltas: [N, 4] where each row is [dy, dx, log(dh), log(dw)]
    """
    # Convert to y, x, h, w
    height   = boxes[:, 2] - boxes[:, 0]
    width    = boxes[:, 3] - boxes[:, 1]
    center_y = boxes[:, 0] + 0.5 * height
    center_x = boxes[:, 1] + 0.5 * width
    
    # Apply deltas
    center_y += deltas[:, 0] * height
    center_x += deltas[:, 1] * width
    height   *= tf.exp(deltas[:, 2])
    width    *= tf.exp(deltas[:, 3])
    
    # Convert back to y1, x1, y2, x2
    y1 = center_y - 0.5 * height
    x1 = center_x - 0.5 * width
    y2 = y1 + height
    x2 = x1 + width
    result = tf.stack([y1, x1, y2, x2], axis=1, name="apply_box_deltas_out")
    return result


def clip_boxes_graph(boxes, window):
    """
    clip refined anchor boxes such that they remain within the dimensions of the image 
    boxes:  [N, 4] each row is y1, x1, y2, x2
    window: [4] in the form y1, x1, y2, x2 
    """
    # Split corners
    wy1, wx1, wy2, wx2 = tf.split(window, 4)   #  0, 0 , 128,128
    y1 , x1 , y2 , x2  = tf.split(boxes, 4, axis=1)
    
    # Clip
    y1 = tf.maximum(tf.minimum(y1, wy2), wy1)   # ensure  wy1 <= y1 <= wy2
    x1 = tf.maximum(tf.minimum(x1, wx2), wx1)
    y2 = tf.maximum(tf.minimum(y2, wy2), wy1)
    x2 = tf.maximum(tf.minimum(x2, wx2), wx1)
    clipped = tf.concat([y1, x1, y2, x2], axis=1, name="clipped_boxes")
    return clipped

def suppress_small_boxes_graph(boxes, scores, area_threshold ):
    """
    supress boxes with area less than area_threshold
    boxes:  [N, 4] each row is y1, x1, y2, x2
    
    """
    bx_area = (boxes[...,2]-boxes[...,0])*(boxes[...,3]-boxes[...,1])
    selected_idxs   = tf.where(tf.greater_equal(bx_area, area_threshold))
    selected_boxes  = tf.gather_nd(boxes, selected_idxs)
    selected_scores = tf.gather_nd(scores, selected_idxs)
    padding   = tf.maximum(tf.shape(boxes)[0] - tf.shape(selected_boxes)[0], 0)

#     print(' box area       : ', bx_area.shape)    
#     print(' selected_idxs  : ', tf.shape(selected_idxs).eval())
#     print(' selected scores: ', tf.shape(selected_scores).eval())
#     print(' Req padding    : ', padding.eval())
    
    selected_boxes  = tf.pad(selected_boxes, [(0, padding), (0, 0)])    
    selected_scores = tf.pad(selected_scores, [(0, padding)])    
    return selected_boxes, selected_scores

def nms(normalized_boxes, scores):
    proposal_count = model.config.POST_NMS_ROIS_TRAINING
    rms_threshold  = model.config.RPN_NMS_THRESHOLD
    indices = tf.image.non_max_suppression(normalized_boxes, 
                                           scores, 
                                           proposal_count,
                                           nms_threshold, 
                                           name="rpn_non_max_suppression")

    proposals = tf.gather(normalized_boxes, indices)
    # Pad if needed
    padding   = tf.maximum(proposal_count - tf.shape(proposals)[0], 0)
    proposals = tf.pad(proposals, [(0, padding), (0, 0)])
    return proposals

In [ ]:
with sess.as_default():
    scores = rpn_class[:, :, 1]
    print(scores.shape)
    deltas = rpn_bbox
    print('deltas shape', deltas.shape)
    deltas = deltas * np.reshape(model.config.RPN_BBOX_STD_DEV, [1, 1, 4])
    print('deltas shape', deltas.shape)
    anchors = model.anchors
    print('model.anchors.shape', model.anchors.shape)
    pre_nms_limit = min(6000, model.anchors.shape[0])
    print('pre nms limit', pre_nms_limit)

    ix = tf.nn.top_k(scores, pre_nms_limit, sorted=True,name="top_anchors").indices

    ## gather top scores (pre_nms_limit = min(6000, # anchors) number of scores from scores)        
    scores  = utils.batch_slice([scores, ix], lambda x, y: tf.gather(x, y), model.config.IMAGES_PER_GPU) 
    print(' selected scores: ',scores.shape)

    ## get corrsponding deltas generated by RPN
    deltas  = utils.batch_slice([deltas, ix], lambda x, y: tf.gather(x, y), model.config.IMAGES_PER_GPU)
    print(' selected deltas: ',deltas.shape)
    anchors = utils.batch_slice(         ix , lambda x   : tf.gather(anchors, x), model.config.IMAGES_PER_GPU, names=["pre_nms_anchors"])
    print(' selected anchors: ',anchors.shape)
    boxes = utils.batch_slice([anchors, deltas],
                              lambda x, y: apply_box_deltas_graph(x, y),model.config.IMAGES_PER_GPU,
                              names=["refined_anchors"])
    print(' delta applied boxes :', boxes.shape)

    # Clip to image boundaries. [batch, N, (y1, x1, y2, x2)]
    height, width = model.config.IMAGE_SHAPE[:2]
    window = np.array([0, 0, height, width]).astype(np.float64)
    print(' window is ', window)
    clipped_boxes  = utils.batch_slice(boxes, 
                                   lambda x: clip_boxes_graph(x, window), model.config.IMAGES_PER_GPU,
                                   names=["refined_anchors_clipped"])

    print(' clipped boxes :', clipped_boxes.shape)
    
    ## Suppress proposal boxes (and  corresponding score) if the area is less than ROI_AREA_THRESHOLD
    roi_area_threshold = 2
    mod_boxes, mod_scores = utils.batch_slice([clipped_boxes,scores], 
                            lambda x, y: suppress_small_boxes_graph(x, y, roi_area_threshold), model.config.IMAGES_PER_GPU,
                            names=["mod_boxes", "mod_scores"])  
    print(' mod boxes :', tf.shape(mod_boxes).eval())
    print(' mod_scores:', tf.shape(mod_scores).eval())   
    
    normalized_boxes = tf.cast(mod_boxes / np.array([[height, width, height, width]]), tf.float32)
    print(' normalized boxes:', normalized_boxes.shape)

    proposals = utils.batch_slice([normalized_boxes, mod_scores], nms, model.config.IMAGES_PER_GPU)
    print('     Output: Prposals shape : ', proposals.shape, tf.shape(proposals).eval())
    

####  Analyze proposals results 

In [ ]:
sess = KB.get_session()
with sess.as_default():
    proposals = tf.identity(rpn_proposal_rois)  # <--- this uses the results from the model 

    bx_area = (proposals[...,2]-proposals[...,0])*(proposals[...,3]-proposals[...,1])
    print(' proposals :', tf.shape(proposals).eval())
    print(' box area : ', tf.shape(bx_area).eval())
    
    selected_idxs = tf.where(tf.less_equal(bx_area, (2/(128*128))) )
    print('selected bx:', tf.shape(selected_idxs).eval())
    
    print(selected_idxs.eval())
    selected_area      = tf.gather_nd(bx_area  , selected_idxs)
    selected_proposals = tf.gather_nd(proposals, selected_idxs)
    print('selected proposals shape', tf.shape(selected_proposals).eval())
    print(selected_proposals[0:30].eval())
    print('selected area shape', tf.shape(selected_area).eval())
    print(selected_area[0:30].eval())
    

####  Analyze bounding box areas 

In [ ]:
with sess.as_default():
    print(' boxes :', boxes.shape)
    for i in [0,10,17,25,26,34,39]:
        print(i, ' non-clipped ', boxes[0, i].eval())
    bx_area = (boxes[...,2]-boxes[...,0])*(boxes[...,3]-boxes[...,1])
    print(' box area : ', bx_area.shape)
    np.set_printoptions(linewidth=130,precision=4,threshold=4096)
    print(bx_area[:, :20].eval(session=sess))
    small_idxs = tf.where(bx_area < 1)
    print('small bx:', tf.shape(small_idxs).eval())
    print(small_idxs[0:10].eval())
    small_area  = tf.gather_nd(bx_area, small_idxs)
    small_boxes = tf.gather_nd(boxes, small_idxs)
    print('small boxes shape', tf.shape(small_boxes).eval())
    print(small_boxes[0:30].eval())
    print('small area shape', tf.shape(small_area).eval())
    print(small_area[0:30].eval())
    

####  Setup tensors to be passed to `detections_target_graph()`    - Detection Target Layer

In [ ]:
import mrcnn.utils  as utils
from mrcnn.detect_tgt_layer import overlaps_graph
# sess = KB.get_session()
# with  sess.as_default():
try:
    sess.close()
    print('session was deleted ')
except:
    print('Session was not defined ')
    pass
sess = tf.InteractiveSession()

image_id = 1

proposals    = KB.identity(rpn_proposal_rois)[image_id]
gt_class_ids = KB.identity(input_gt_class_ids)[image_id]
gt_boxes     = KB.cast(KB.identity(input_gt_bboxes_norm), dtype='float32')[image_id]
# gt_masks     = KB.identity(input_gt_masks)
print('rpn_roi_proposals')
print(proposals.dtype, gt_class_ids.dtype, gt_boxes.dtype)
print(proposals.shape)
print(proposals.eval())
print('gt_class_ids')
print(gt_class_ids.shape)
print(gt_class_ids.eval())
print('gt_boxes')
print(gt_boxes.shape)
print(gt_boxes.eval())
# proposals    = rpn_proposal_rois[1]
# gt_class_ids = input_gt_class_ids[1]
# gt_boxes     = input_normlzd_gt_boxes[1]
# gt_masks     = input_gt_masks[1]
# config       = model.config